## !pip install --upgrade pip
## !pip install SQLAlchemy==1.3
## !pip install pyhive[hive]
## !pip install thrift
## !pip install sasl
## !pip install thrift-sasl 
## !pip install psycopg2
## 
## 
## /**
## 注意：
## @date1表示开始时间，@date2表示结束时间，一般两者相差一天，即取一天的数据。
## bd/ad/ld/cp格式都是yyyymmddhh24，比如2023032301
## @batchId表示批次的变量，yyyymmddhh24格式，比如2023032301
## udf_is_null 自定义函数，用于判断字段是否为空，如果字段为空/null或者空字符串，都返回0，否则返回1
## --主表建表语句 TABLE tb_test --每天1T数据，f02本端号码（去重后2千万），f04对端号码（去重后2千万），f16中文内容
## */


In [ ]:
from sqlalchemy.engine import create_engine
import pandas as pd

engine = create_engine("hive://spark-thrift:10000/default")
pd.read_sql_query("create database IF NOT EXISTS sample;", con=engine)
pd.read_sql_query("use sample;", con=engine)
pd.read_sql_query("show tables;", con=engine)


In [ ]:
pd.read_sql_query("""drop table if exists tb_test""", con=engine)
pd.read_sql_query("""drop table if exists tb_sev_u""", con=engine)
pd.read_sql_query("""drop table if exists tb_test_qf_stat""", con=engine)

pd.read_sql_query("""
CREATE TABLE tb_test
(
f01    BIGINT comment 'unix_timestamp',
f02    string comment 'f02本端号码（去重后2千万）',
f03    string,
f04    string comment 'f04对端号码（去重后2千万）, count distinct clause',
f05    string,
f06    string comment 'group by clause, ref-a',
f07    string comment 'group by clause, ref-b',
f08    string,
f09    string,
f10    INT,
f11    string,
f12    string,
f13    string,
f14    string comment 'data filtering col, where f14 = 49',
f15    BIGINT,
f16    string comment '中文文本内容，需要制表符转变',
f17    string,
f18    string,
f19    INT,
f20    string,
f21    string,
f22    INT comment '1st group by clause, maybe category',
f23    string,
f24    string,
f25    string,
f26    string,
f27    string,
f28    string,
f29    string,
f30    bigint comment 'business datetime',
f31    bigint comment 'processing datetime'
) partitioned by (cp bigint, ld bigint) 
stored as orc tblproperties ("orc.compress"="ZLIB");
""", con=engine)

pd.read_sql_query("""
create table tb_sev_u (id string, name string);
""", con=engine)

pd.read_sql_query("""
create table if not exists tb_test_qf_stat(
f22 string,
f02 string,
f16 string,
cnt bigint,
f06    string,
f07    string
)partitioned by (bd string, ad bigint);
""", con=engine)

In [ ]:
pd.read_sql_query("show tables;", con=engine)

造数支持SQL Query测试

In [ ]:
cp = 2023032401
ld = 2023032401
pd.read_sql_query("""
insert into tb_test partition(cp={cp}, ld={ld})
values
(1683088297 , 'S15217958191' ,  'A123456' , 'S15217958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容' , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088397 , 'S15217958192' , 'A133456' , 'S15317958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容' , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088497 , 'S15217958193' , 'A143456' , 'S15417958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3' , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088597 , 'S15217958194' , 'A153456' , 'S15517958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容' , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S15217958195' , 'A163456' , 'S15617958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容' , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15617958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15627958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15637958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15647958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15657958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15627958192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15637158192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15647258192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15657358192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15627458192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15637558192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15647658192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)
, (1683088697 , 'S25217958195' , 'A163456' , 'S15657758192' , 'SAA' , 'SBB' , 'SCC' , 'SDD' , 'SEE' , 12345 , 'SFF' , 'SGG' , 'SHH' , 49 , 123456789 , 'S中文文本内容*1中文文本内容*2中文文本内容*3'  , 'SII' , 'SJJ' , 123 , 'SLL' , 'SUU' , 234 , 'SWW' , 'SXX' , 'SYY' , 'SZZ' , 'SAAAA' , 'SABAB' , 'SACAC' , 2023032301 , 2023032301)

""".format(cp=cp, ld=ld), con=engine)

pd.read_sql("""select * from tb_test""", con=engine)

In [ ]:
pd.read_sql_query("""
insert into tb_sev_u
values ('S15217958191','AA-BBBBBB-CCCCCC-1' )
, ('S15217958192','AA-BBBBBB-CCCCCC-2' )
, ('S15217958193','AA-BBBBBB-CCCCCC-3' )
, ('S15217958194','AA-BBBBBB-CCCCCC-4' )
, ('S15217958195','AA-BBBBBB-CCCCCC-5' )
""", con=engine)

pd.read_sql("""select * from tb_sev_u""", con=engine)

增量日志清洗输出给数仓,之前要设置 Hive 支持动态分区。

In [ ]:
pd.read_sql_query("""set hive.exec.dynamic.partition=true;""", con=engine)

In [ ]:
pd.read_sql_query("""set hive.exec.dynamic.partition.mode=nonstrict;""", con=engine)

In [ ]:
batchId = 2023032301
date1 = 2023032301
date2 = 2023032324

pd.read_sql_query("alter table tb_test_qf_stat drop if exists partition (ad = {batchId})".format(batchId=batchId), con=engine)

query = """
with tb_test_qf_tmp as (
    select trim(f02) as f02, trim(f04) as f04,trim(f22) as f22,
    regexp_replace(regexp_replace(regexp_replace(trim(f16),'\\\?+','\?') ,'0+','0'),'[ \t]+',' ') as f16,
    from_unixtime(unix_timestamp(cast(f30 as string),'yyyyMMddhh'), 'yyyy-MM-dd')  as bd, 
    {batchId} as ad,
    trim(f06) as f06,trim(f07) as f07
    from tb_test t
    where f31 >= {date1} and f31 < {date2} and f14 = '49'
),
tb_test_qf_tmp1 as (
select a.f22,a.f02,a.f16, count(distinct a.f04) as cnt,a.bd ,a.ad,
a.f06,a.f07
from tb_test_qf_tmp a
left join tb_sev_u b on a.f02 = b.id
where a.f02 is not null and b.id is null
and length(a.f16) > 10
group by a.f22,a.f02,a.f16,a.bd,a.ad,a.f06,a.f07 having cnt > 10 
)
insert into tb_test_qf_stat partition(bd, ad)
select f22,f02,f16,cnt,f06,f07,bd,ad
from tb_test_qf_tmp1
""".format(batchId=batchId, date1=date1, date2=date2)

pd.set_option('display.max_colwidth', None)
df = pd.read_sql_query("explain FORMATTED  "+ query, con=engine)
print(df)

pd.read_sql_query(query, con=engine)

In [ ]:
query = """
select f22,f02,f16,cnt,bd,f06,f07 from
( select *, row_number() over(partition by f02 order by bd desc ) rn from tb_test_qf_stat) t
where t.rn =1;
"""
pd.read_sql_query(query, con=engine)


In [ ]:
pd.read_sql_query(query, con=engine)

使用 hive 外表读文字

In [ ]:
pd.read_sql_query("""
create table ext_tb_sev_u (big5context string) location '/user/hive/external/';
""", con=engine)

In [ ]:
 pd.read_sql_query("""
 with tmp as (
 select right(big5context, cast((1+rand()*50) as int)) as f16 from ext_tb_sev_u where big5context is not null and length(trim(big5context)) > 0 
 ) ,
tmp1 as (
 select
 regexp_replace(regexp_replace(regexp_replace(trim(f16),'\\\?+','\?') ,'0+','0'),'[ \t]+',' ') as ftxt
 from tmp
 )
 select *
 from tmp1
 ;""", con=engine)

In [ ]:
pd.read_sql_query("""select cast (least(cast(current_timestamp as int), 946656000+cast((1+rand()*1000000000) as int)) as timestamp) as unix_ts ;""", con=engine)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


In [ ]:
import pandas as pd
import numpy as np
batch_size = 10*1000
LENGTH = 10

df = pd.DataFrame()

cols=["f01", "f02","f03", "f04", "f05", "f06", "f07", "f08", "f09", "f10","f11","f12","f13","f14","f15","f16","f17","f18","f19","f20","f21","f22","f23","f24","f25","f26","f27","f28","f29","f30","f31" ]
random_txtcols=["f03","f05","f08","f09","f11","f12","f13","f16","f17","f18","f20","f21","f23","f24","f25","f26","f27","f28","f29"]
alphabet = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

current_ts = int(pd.to_datetime(pd.Timestamp.now()).value/10**9)
df2 = pd.DataFrame(np.random.randint(19200000000,19200000000+20000000,size=(batch_size,2)), columns=list(["f02", "f04"])) 
df1 = pd.DataFrame(np.random.randint(946656000, current_ts, size=(batch_size,1)),  columns=list(["f01"]))
df6 = pd.DataFrame(np.random.randint(0, 10, size=(batch_size,5)),  columns=list(["f06", "f07", "f10", "f15", "f19"]))
df14 = pd.DataFrame(np.random.randint(48, 51, size=(batch_size,1)),  columns=list(["f14"]))
df22 = pd.DataFrame(np.random.randint(1, 10, size=(batch_size,1)),  columns=list(["f22"]))
df30 = pd.DataFrame(np.random.randint(current_ts-(30*24*3600), current_ts, size=(batch_size,2)),  columns=list(["f30","f31"]))

for k in random_txtcols:
    np_batchsize = np.random.choice(np.array(alphabet, dtype="|U1"), [batch_size, LENGTH])
    df[k] = pd.DataFrame( ["".join(np_batchsize[i]) for i in range(len(np_batchsize))], columns=[k])


df = pd.concat([df1,df2, df6, df14, df22, df30, df], axis=1, join='inner')
df = df[cols]
df


In [ ]:
current_ts = int(pd.to_datetime(pd.Timestamp.now()).value/10**9)
current_ts